In [ ]:
import json
import os
import openai
from pathlib import Path

# TODO: Add instructions for API key setup, e.g.,
# openai.api_key = "YOUR_OPENAI_API_KEY"
# or set the OPENAI_API_KEY environment variable.

def transcribe_audio(file_path: Path) -> dict:
    """
    Transcribes an audio file using the OpenAI Whisper API.

    Args:
        file_path: Path to the audio file (e.g., MP4).

    Returns:
        A dictionary containing the transcription result from the API.
    """
    print(f"Transcribing audio file: {file_path}...")
    with open(file_path, "rb") as audio_file:
        transcription = openai.Audio.transcribe(
            model="whisper-1",
            file=audio_file,
            response_format="verbose_json"  # Request verbose JSON for timestamps
        )
    print("Transcription complete.")
    return transcription

def format_transcription_to_markdown(transcription_result: dict) -> str:
    """
    Formats the verbose JSON transcription into a markdown string with timestamps.

    Args:
        transcription_result: The verbose JSON response from the Whisper API.

    Returns:
        A string formatted in markdown with timestamps.
    """
    formatted_lines = []
    if "segments" in transcription_result:
        for segment in transcription_result["segments"]:
            start_time = segment["start"]
            end_time = segment["end"]
            text = segment["text"]
            # Format timestamps (e.g., HH:MM:SS.mmm)
            start_ts = f"{int(start_time // 3600):02}:{int((start_time % 3600) // 60):02}:{start_time % 60:06.3f}"
            end_ts = f"{int(end_time // 3600):02}:{int((end_time % 3600) // 60):02}:{end_time % 60:06.3f}"
            formatted_lines.append(f"[{start_ts} -> {end_ts}] {text.strip()}")
    else:
        # Fallback for non-segmented transcription (though verbose_json should have segments)
        formatted_lines.append(transcription_result.get("text", "No transcription text found."))

    return "\n".join(formatted_lines)

def main():
    # --- Configuration ---
    # TODO: Replace with the actual path to your MP4 file
    input_mp4_path = Path("_input_data/sample.mp4")
    # TODO: Create a dummy _input_data/sample.mp4 for initial testing if it doesn't exist
    # For example, by creating an empty file or a very short silent mp4.
    # Ensure the _input_data directory exists. Path(input_mp4_path.parent).mkdir(parents=True, exist_ok=True)

    # Ensure output directory exists
    output_dir = Path("_input_data")
    output_dir.mkdir(parents=True, exist_ok=True)
    
    output_transcription_path_md = output_dir / f"{input_mp4_path.stem}_transcription.md"
    output_transcription_path_txt = output_dir / f"{input_mp4_path.stem}_transcription.txt"

    # --- Create a dummy input file for initial structure testing ---
    # This is important so the notebook can run end-to-end without a real MP4 initially.
    # The user will replace this with their actual MP4 file.
    if not input_mp4_path.exists():
        print(f"Creating dummy file: {input_mp4_path}")
        input_mp4_path.parent.mkdir(parents=True, exist_ok=True)
        with open(input_mp4_path, "w") as f:
            f.write("This is a dummy MP4 file for testing purposes.")
        print(f"Dummy file {input_mp4_path} created. Replace it with your actual MP4 file.")


    # --- Check for API Key ---
    if not os.getenv("OPENAI_API_KEY") and not openai.api_key:
        print("OpenAI API key not found. Please set the OPENAI_API_KEY environment variable or set openai.api_key directly in the notebook.")
        print("Skipping transcription.")
        return

    # --- Transcription ---
    try:
        transcription_result = transcribe_audio(input_mp4_path)
    except Exception as e:
        print(f"An error occurred during transcription: {e}")
        # Create an empty transcription result if API call fails (e.g. dummy file)
        transcription_result = {"segments": [{"start": 0, "end": 0, "text": f"Error during transcription: {e}. Ensure your input file is a valid audio/video file."}]}


    # --- Formatting ---
    formatted_text_md = format_transcription_to_markdown(transcription_result)
    
    # For a plain text file, the markdown format is already quite readable.
    # If a different format is needed for .txt, a separate function could be made.
    formatted_text_txt = formatted_text_md 

    # --- Output ---
    print("\n--- Formatted Transcription (Markdown) ---")
    print(formatted_text_md)

    # Save to Markdown file
    with open(output_transcription_path_md, "w", encoding="utf-8") as f:
        f.write(formatted_text_md)
    print(f"\nTranscription saved to: {output_transcription_path_md}")

    # Save to Text file
    with open(output_transcription_path_txt, "w", encoding="utf-8") as f:
        f.write(formatted_text_txt)
    print(f"Transcription saved to: {output_transcription_path_txt}")


if __name__ == "__main__":
    # This block allows the script to be run from the command line as well,
    # though it's primarily designed for a Jupyter notebook.
    # In a notebook, you would typically call main() in a cell.
    main()
